In [2]:
import networkx as nx
import pandas as pd
import pickle
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt 
import seaborn as sns
import scipy.spatial.distance as spsd
import scipy.linalg as spla
import sys
import random
import math
import itertools

In [3]:
with open("/Users/app/Desktop/科研资料/With Prof/Prof Qin/Networ Sampling/Variable/name_list.pkl",'rb') as f:
    Name_List=pickle.load(f)
Name_List.remove("FFS_Prob")
Name_List.remove("Induced_FFS_Prob")


In [11]:
path='/Users/app/Desktop/科研资料/With Prof/Prof Qin/Networ Sampling/result_csv/real_data'
result=list(Path(path).glob("*"+'.csv'))
#result=list(Path('/Users/app/Desktop/MaxD/D_d/beta_1000').glob("*"+'.pkl'))
#
result=sorted(result)
len(result)
def style_specific_cell(x):
    global color_pos 
    color = 'background-color: red'
    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
    for x in color_pos:
        df1.iloc[x[0],x[1]] = color
    return df1

In [5]:
result

[PosixPath('/Users/app/Desktop/科研资料/With Prof/Prof Qin/Networ Sampling/result_csv/real_data/bitcoin.csv'),
 PosixPath('/Users/app/Desktop/科研资料/With Prof/Prof Qin/Networ Sampling/result_csv/real_data/coauthor.csv'),
 PosixPath('/Users/app/Desktop/科研资料/With Prof/Prof Qin/Networ Sampling/result_csv/real_data/email.csv'),
 PosixPath('/Users/app/Desktop/科研资料/With Prof/Prof Qin/Networ Sampling/result_csv/real_data/lastfm.csv'),
 PosixPath('/Users/app/Desktop/科研资料/With Prof/Prof Qin/Networ Sampling/result_csv/real_data/p2p.csv')]

In [6]:
R=[pd.read_csv(result[3]),pd.read_csv(result[4]),pd.read_csv(result[1]),pd.read_csv(result[0]),pd.read_csv(result[2])]
#lastfm p2p coauthor bitcoin email
frame=pd.DataFrame(R[0].iloc[:,0])
frame=frame.rename(columns={R[0].columns[0]: "Method"})
frame.head()

,Method
0,RN
1,DN
2,PN
3,RE
4,Induced_RE


In [82]:
j=11
df1=pd.concat([x.iloc[:,j] for x in R], axis=1)

df=pd.concat([frame,df1], axis=1)
print(R[0].columns[j])
#lastfm p2p coauthor bitcoin email
df.columns=['Method','lastfm_L2','p2p_L2','coauthor_L2','bitcoin_L2','email_L2']
df.head()

LCC L2 Distribution


,Method,lastfm_L2,p2p_L2,coauthor_L2,bitcoin_L2,email_L2
0,RN,110.563047,1.411322,459.713792,68.979339,149.360006
1,DN,17.756185,1.435438,79.033389,73.330453,19.879886
2,PN,4.890847,0.436859,310.246288,36.749686,8.406240
3,RE,149.856930,1.622495,584.842811,88.636592,521.598261
4,Induced_RE,25.871513,1.694167,34.419796,75.975723,24.663439


In [95]:
## gcc 8

j=24
df1=pd.concat([x.iloc[:,j] for x in R], axis=1)
df2=pd.concat([x.iloc[:,j+1] for x in R], axis=1)#*10**3
df3=pd.concat([x.iloc[:,j+2] for x in R], axis=1)#10 **7 #10**4 for Mod Var Gcc

df=pd.concat([frame,df1,df2,df3], axis=1)
print(R[0].columns[j],'\n',R[0].columns[j+1],'\n',R[0].columns[j+2])
df.columns=['Method','lastfm_NMSE','p2p_NMSE','coauthor_NMSE','bitcoin_NMSE','email_NMSE','lastfm_Bias','p2p_Bias','coauthor_Bias','bitcoin_Bias','email_Bias','lastfm_Variance','p2p_Variance','coauthor_Variance','bitcoin_Variance','email_Variance']
df.head()
#df=pd.concat([df+[x.iloc[:,2] for x in R]], axis=1)

2nd largest eigenvalue nmse 
 2nd largest eigenvalue bias 
 2nd largest eigenvalue var


,Method,lastfm_NMSE,p2p_NMSE,coauthor_NMSE,bitcoin_NMSE,email_NMSE,lastfm_Bias,p2p_Bias,coauthor_Bias,bitcoin_Bias,email_Bias,lastfm_Variance,p2p_Variance,coauthor_Variance,bitcoin_Variance,email_Variance
0,RN,0.417227,1.124470,0.215208,0.686410,0.591465,0.001494,0.002559,-0.000115,0.004176,0.020631,7.367961e-07,3.547922e-07,2.421082e-06,4.312772e-06,0.000047
1,DN,3.999081,3.713390,4.665832,5.225920,1.623968,0.016405,0.008653,0.033555,0.035461,0.059099,3.496879e-06,3.959909e-07,2.545775e-05,3.155537e-06,0.000070
2,PN,2.876110,2.791578,0.705702,4.522207,1.424109,0.011799,0.006491,0.004357,0.030657,0.051701,1.803470e-06,3.982797e-07,7.386221e-06,4.109577e-06,0.000067
3,RE,0.273915,0.434890,0.426858,0.153548,0.556621,-0.001114,0.000989,-0.003089,0.000885,-0.020397,3.748397e-08,5.513580e-08,9.705737e-08,3.033919e-07,0.000002
4,Induced_RE,4.135381,3.749641,4.800179,5.243304,1.733399,0.016940,0.008732,0.034366,0.035576,0.062971,4.553816e-06,4.899412e-07,3.758843e-05,3.374871e-06,0.000094


In [16]:
#delete induced method
df_drop=df[~df.Method.str.contains("Induced")]
df= df_drop.reset_index(drop=True)

In [72]:
#drop edge based : density 
df_induced=df[df.Method.str.contains("Induced")]
#dF.iloc[95:, np.r_[0:3, -1]]
df_node_induced=pd.concat([df.iloc[0:3,:],df_induced],axis=0)
#df_node_induced=pd.concat([df_node_induced,df.iloc[-1,:]],axis=0)

#df = df_node_induced.reset_index(drop=True)

#df.iloc[np.r_[0:3,-1],:]

In [75]:
#df_node_induced.append(df.iloc[-1,:].values)

In [96]:
color_pos=[]
for x in range(1,len(df.columns)):
    color_pos.append([df.iloc[list(df.index),x].abs().sort_values().index[0:3],x])
df_=df.style.apply(style_specific_cell, axis=None)
df_.set_precision(5)
display(df_)

,Method,lastfm_NMSE,p2p_NMSE,coauthor_NMSE,bitcoin_NMSE,email_NMSE,lastfm_Bias,p2p_Bias,coauthor_Bias,bitcoin_Bias,email_Bias,lastfm_Variance,p2p_Variance,coauthor_Variance,bitcoin_Variance,email_Variance
0,RN,0.41723,1.12447,0.21521,0.68641,0.59147,0.00149,0.00256,-0.00011,0.00418,0.02063,0.00000,0.00000,0.00000,0.00000,0.00005
1,DN,3.99908,3.71339,4.66583,5.22592,1.62397,0.01641,0.00865,0.03355,0.03546,0.05910,0.00000,0.00000,0.00003,0.00000,0.00007
2,PN,2.87611,2.79158,0.70570,4.52221,1.42411,0.01180,0.00649,0.00436,0.03066,0.05170,0.00000,0.00000,0.00001,0.00000,0.00007
3,RE,0.27392,0.43489,0.42686,0.15355,0.55662,-0.00111,0.00099,-0.00309,0.00089,-0.02040,0.00000,0.00000,0.00000,0.00000,0.00000
4,Induced_RE,4.13538,3.74964,4.80018,5.24330,1.73340,0.01694,0.00873,0.03437,0.03558,0.06297,0.00000,0.00000,0.00004,0.00000,0.00009
5,RNE,0.39463,0.18964,0.57866,0.11325,0.60170,-0.00161,0.00042,-0.00420,-0.00035,-0.02208,0.00000,0.00000,0.00000,0.00000,0.00000
6,Induced_RNE,1.92251,1.94055,0.46640,3.41709,1.22066,0.00784,0.00449,0.00266,0.02314,0.04406,0.00000,0.00000,0.00000,0.00000,0.00007
7,HYB,0.37712,0.21043,0.56028,0.09690,0.59667,-0.00154,0.00047,-0.00407,-0.00014,-0.02189,0.00000,0.00000,0.00000,0.00000,0.00000
8,Induced_HYB,2.41481,2.37830,1.40584,3.80547,1.31748,0.00985,0.00552,0.00937,0.02578,0.04762,0.00000,0.00000,0.00002,0.00000,0.00008
9,SBS,0.03390,1.07183,0.21856,0.15996,0.24581,-0.00014,0.00250,-0.00159,0.00109,-0.00903,0.00000,0.00000,0.00000,0.00000,0.00000


In [45]:

j=6
#df1=pd.concat([x.iloc[:,j] for x in R], axis=1)
#df2=pd.concat([x.iloc[:,j+1]*10**3 for x in R], axis=1)#*10**3
df3=pd.concat([x.iloc[:,j]*10**5 for x in R], axis=1)#10 **7 #10**4 for Mod Var Gcc

df=pd.concat([frame,df3], axis=1)
print(df3.columns[0])
#print(R[0].columns[j],'\n',R[0].columns[j+1],'\n',R[0].columns[j+2])
df.columns=['Method','ER_Variance','SBM_Variance','WS_Variance','BA_Variance','Graphon_Variance']
df.head()
#df=pd.concat([df+[x.iloc[:,2] for x in R]], axis=1)

density var


,Method,ER_Variance,SBM_Variance,WS_Variance,BA_Variance,Graphon_Variance
0,RN,0.002088,0.008104,0.000142,0.000977,0.007246
1,DN,0.002032,0.016165,0.000155,0.004072,0.007763
2,PN,0.002103,0.010278,0.000137,0.004117,0.007535
3,RE,0.000006,0.000006,0.000005,0.000015,0.000006
4,Induced_RE,0.001845,0.025514,0.000149,0.003654,0.008215


In [56]:
df_induced=df[df.Method.str.contains("Induced")]
df_node_induced=pd.concat([df.iloc[0:3,:],df_induced],axis=0)
df_node_induced = df_node_induced.reset_index(drop=True)
df_node_induced

,Method,ER_Variance,SBM_Variance,WS_Variance,BA_Variance,Graphon_Variance
0,RN,0.002088,0.008104,0.000142,0.000977,0.007246
1,DN,0.002032,0.016165,0.000155,0.004072,0.007763
2,PN,0.002103,0.010278,0.000137,0.004117,0.007535
3,Induced_RE,0.001845,0.025514,0.000149,0.003654,0.008215
4,Induced_RNE,0.001795,0.013035,0.000167,0.003265,0.007988
5,Induced_HYB,0.001903,0.016482,0.000158,0.003081,0.008090
6,Induced_SBS,0.002032,1.250510,0.000531,0.003269,0.008624
7,Induced_FFS_Geom,0.001795,1.339440,0.000487,0.003153,0.008855
8,Induced_FFS_Prob,0.001954,0.214305,0.000450,0.003331,0.008306
9,Induced_BFS,0.001966,2.702637,0.000427,0.003534,0.086365


In [58]:
color_pos=[]
for x in range(1,len(df_node_induced.columns)):
    color_pos.append([df_node_induced.iloc[list(df_node_induced.index),x].abs().sort_values().index[0:3],x])
df_node_induced=df_node_induced.style.apply(style_specific_cell, axis=None)
df_node_induced.set_precision(5)
display(df_node_induced)

,Method,ER_Variance,SBM_Variance,WS_Variance,BA_Variance,Graphon_Variance
0,RN,0.00209,0.00810,0.00014,0.00098,0.00725
1,DN,0.00203,0.01616,0.00015,0.00407,0.00776
2,PN,0.00210,0.01028,0.00014,0.00412,0.00753
3,Induced_RE,0.00185,0.02551,0.00015,0.00365,0.00822
4,Induced_RNE,0.00180,0.01304,0.00017,0.00327,0.00799
5,Induced_HYB,0.00190,0.01648,0.00016,0.00308,0.00809
6,Induced_SBS,0.00203,1.25051,0.00053,0.00327,0.00862
7,Induced_FFS_Geom,0.00180,1.33944,0.00049,0.00315,0.00885
8,Induced_FFS_Prob,0.00195,0.21431,0.00045,0.00333,0.00831
9,Induced_BFS,0.00197,2.70264,0.00043,0.00353,0.08637


In [ ]:
with open("/Users/app/Desktop/科研资料/With Prof/Prof Qin/Networ Sampling/Variable/name_list.pkl",'rb') as f:
    Name_List=pickle.load(f)

path='/Users/app/Desktop/科研资料/With Prof/Prof Qin/Networ Sampling/result_csv'
result=list(Path(path).glob("*"+'.csv'))
#result=list(Path('/Users/app/Desktop/MaxD/D_d/beta_1000').glob("*"+'.pkl'))
#
result=sorted(result)
len(result)


result

R=[pd.read_csv(result[1]),pd.read_csv(result[3]),pd.read_csv(result[4]),pd.read_csv(result[0]),pd.read_csv(result[2])]
#ER  SBM WS BA Graphon
frame=pd.DataFrame(R[0].iloc[:,0])
frame=frame.rename(columns={R[0].columns[0]: "Method"})
frame.head()

j=11
df1=pd.concat([x.iloc[:,j] for x in R], axis=1)

df=pd.concat([frame,df1], axis=1)
print(R[0].columns[j])
df.columns=['Method','ER_L2','SBM_L2','WS_L2','BA_L2','Graphon_L2']
df.head()

## gcc 8

j=1
df1=pd.concat([x.iloc[:,j] for x in R], axis=1)
df2=pd.concat([x.iloc[:,j+1] for x in R], axis=1)#*10**3
df3=pd.concat([x.iloc[:,j+2] for x in R], axis=1)#10 **7 #10**4 for Mod Var Gcc

df=pd.concat([frame,df1,df2,df3], axis=1)
print(R[0].columns[j],'\n',R[0].columns[j+1],'\n',R[0].columns[j+2])
df.columns=['Method','ER_NMSE','SBM_NMSE','WS_NMSE','BA_NMSE','Graphon_NMSE','ER_Bias','SBM_Bias','WS_Bias','BA_Bias','Graphon_Bias','ER_Variance','SBM_Variance','WS_Variance','BA_Variance','Graphon_Variance']
df.head()
#df=pd.concat([df+[x.iloc[:,2] for x in R]], axis=1)

#delete induced method
df_drop=df[~df.Method.str.contains("Induced")]
df= df_drop.reset_index(drop=True)

#drop edge based : density 
df_induced=df[df.Method.str.contains("Induced")]
df_node_induced=pd.concat([df.iloc[[0:3,-1],:],df_induced],axis=0)
df = df_node_induced.reset_index(drop=True)


color_pos=[]
for x in range(1,len(df.columns)):
    color_pos.append([df.iloc[list(df.index),x].abs().sort_values().index[0:3],x])
df_=df.style.apply(style_specific_cell, axis=None)
df_.set_precision(5)
display(df_)


j=6
#df1=pd.concat([x.iloc[:,j] for x in R], axis=1)
#df2=pd.concat([x.iloc[:,j+1]*10**3 for x in R], axis=1)#*10**3
df3=pd.concat([x.iloc[:,j]*10**5 for x in R], axis=1)#10 **7 #10**4 for Mod Var Gcc

df=pd.concat([frame,df3], axis=1)
print(df3.columns[0])
#print(R[0].columns[j],'\n',R[0].columns[j+1],'\n',R[0].columns[j+2])
df.columns=['Method','ER_Variance','SBM_Variance','WS_Variance','BA_Variance','Graphon_Variance']
df.head()
#df=pd.concat([df+[x.iloc[:,2] for x in R]], axis=1)

df_induced=df[df.Method.str.contains("Induced")]
df_node_induced=pd.concat([df.iloc[0:3,:],df_induced],axis=0)
df_node_induced = df_node_induced.reset_index(drop=True)
df_node_induced

color_pos=[]
for x in range(1,len(df_node_induced.columns)):
    color_pos.append([df_node_induced.iloc[list(df_node_induced.index),x].abs().sort_values().index[0:3],x])
df_node_induced=df_node_induced.style.apply(style_specific_cell, axis=None)
df_node_induced.set_precision(5)
display(df_node_induced)

